In [1]:
import sys
sys.path.append("/home/mount/project")

In [2]:
from config_sing import *

## get motifs

In [3]:
TARGET = "target_PER1"
sam    = "Input1_20x"
fdiry  = os.path.join(FD_RES, "annotation_fragment", sam, TARGET)
fname  = "*_merge.bed.gz"
fpath  = os.path.join(fdiry, fname)
MOTIFS = np.sort([os.path.basename(fp) for fp in glob(fpath)])
MOTIFS[:3]

array(['AHR_merge.bed.gz', 'AIRE_merge.bed.gz', 'AP1_1_merge.bed.gz'],
      dtype='<U26')

In [4]:
gen = it.combinations(MOTIFS[:6], 2)
for x in gen:
    print(x)

('AHR_merge.bed.gz', 'AIRE_merge.bed.gz')
('AHR_merge.bed.gz', 'AP1_1_merge.bed.gz')
('AHR_merge.bed.gz', 'AP1_2_merge.bed.gz')
('AHR_merge.bed.gz', 'ARI5A_merge.bed.gz')
('AHR_merge.bed.gz', 'ARI5B_merge.bed.gz')
('AIRE_merge.bed.gz', 'AP1_1_merge.bed.gz')
('AIRE_merge.bed.gz', 'AP1_2_merge.bed.gz')
('AIRE_merge.bed.gz', 'ARI5A_merge.bed.gz')
('AIRE_merge.bed.gz', 'ARI5B_merge.bed.gz')
('AP1_1_merge.bed.gz', 'AP1_2_merge.bed.gz')
('AP1_1_merge.bed.gz', 'ARI5A_merge.bed.gz')
('AP1_1_merge.bed.gz', 'ARI5B_merge.bed.gz')
('AP1_2_merge.bed.gz', 'ARI5A_merge.bed.gz')
('AP1_2_merge.bed.gz', 'ARI5B_merge.bed.gz')
('ARI5A_merge.bed.gz', 'ARI5B_merge.bed.gz')


## import data

In [6]:
#fpath = "/home/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1/AP1_1_merge.bed.gz"

TARGET = "target_PER1"
#sam    = "Input1_20x"
sam    = "TFX2_DMSO"
motif_pair = ("AP1_1_merge.bed.gz", "AP1_2_merge.bed.gz")
THRESHOLD_MOTIF = 0

### INIT
lst_dat = []

for motif in motif_pair:
    ### INIT
    lst   = []
    idx   = -1 # HANDLE EXCEPTION: Empty file
    mtf   = motif.replace("_merge.bed.gz", "")
    
    ### set input file path
    fdiry = os.path.join(FD_RES, "annotation_fragment", sam, TARGET)
    fname = motif
    fpath = os.path.join(fdiry, fname)
    
    ### import data
    with gzip.open(fpath, 'rb') as file:
        for idx, line in enumerate(file):

            ### preprocess each line 
            line = str(line, 'utf-8')
            line = line.strip().split("\t")

            ### extract needed values
            idx1, idx2, idx3, idx4 = 5, 6, 8, 9
            len_mtf = int(line[idx2]) - int(line[idx1])
            len_lap = int(line[idx4])
            score   = float(line[idx3])

            ### filtering motifs scores and make sure
            ### fragments cover the full motif
            if (len_mtf <= len_lap) & (score >= THRESHOLD_MOTIF):
                lst.append(line)
    
    ### HANDLE EXCEPTION: Empty data
    if len(lst) == 0:
        if verbose:
            print(mtf, sam, "Skip Empty", flush=True) 
        break

    ### wrap up a list of lines into a dataframe
    CNAMES = ["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
              "Chrom_MTF",  "Start_MTF",  "End_MTF",
              "Motif",      "Score",      "Overlap"]
    CTYPES = [np.str, np.int,   np.int, np.int,
              np.str, np.int,   np.int,
              np.str, np.float, np.int]
    dat = pd.DataFrame(lst, columns=CNAMES) \
        .astype(dict(zip(CNAMES, CTYPES))) \
        .assign(Sample = sam)
    
    ###
    dat = dat.loc[:,["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Sample", "Motif"]]
    
    print(mtf, "Filter", str(idx + 1) + "-" + str(len(lst)), flush=True)        
    lst_dat.append(dat)

AP1_1 Filter 400-398
AP1_2 Filter 575-559


In [7]:
df1, df2 = lst_dat
print(df1.head())
print(df2.head())

  Chrom_Frag  Start_Frag  End_Frag  Count_Frag     Sample  Motif
0      chr17     8148000   8148888           1  TFX2_DMSO  AP1/1
1      chr17     8148001   8148888           1  TFX2_DMSO  AP1/1
2      chr17     8148049   8148884           1  TFX2_DMSO  AP1/1
3      chr17     8148049   8148888           1  TFX2_DMSO  AP1/1
4      chr17     8148050   8148887           1  TFX2_DMSO  AP1/1
  Chrom_Frag  Start_Frag  End_Frag  Count_Frag     Sample  Motif
0      chr17     8148000   8148888           1  TFX2_DMSO  AP1/2
1      chr17     8148001   8148888           1  TFX2_DMSO  AP1/2
2      chr17     8148049   8148884           1  TFX2_DMSO  AP1/2
3      chr17     8148049   8148888           1  TFX2_DMSO  AP1/2
4      chr17     8148050   8148887           1  TFX2_DMSO  AP1/2


**Test merge**

In [14]:
dat = pd.merge(df1, df2, on=['Chrom_Frag', 'Start_Frag', "End_Frag", "Count_Frag", "Sample"], how='outer')
dat = dat.assign(Motif = lambda x: x.Motif_x.astype(str) + "_" + x.Motif_y.astype(str))
dat = dat.assign(Motif = lambda x: x.Motif.str.replace("_nan|nan_", ""))

dat = dat.assign(Group = lambda x: x.Sample.str.replace("[0-9]|_20x", ""))
dat = dat.assign(X     = lambda x: x.Group.astype(str) + "_" + x.Motif.astype(str))

dat.head()

,Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Sample,Motif_x,Motif_y,Motif,Group,X
0,chr17,8148000,8148888,1,TFX2_DMSO,AP1/1,AP1/2,AP1/1_AP1/2,TFX_DMSO,TFX_DMSO_AP1/1_AP1/2
1,chr17,8148001,8148888,1,TFX2_DMSO,AP1/1,AP1/2,AP1/1_AP1/2,TFX_DMSO,TFX_DMSO_AP1/1_AP1/2
2,chr17,8148049,8148884,1,TFX2_DMSO,AP1/1,AP1/2,AP1/1_AP1/2,TFX_DMSO,TFX_DMSO_AP1/1_AP1/2
3,chr17,8148049,8148888,1,TFX2_DMSO,AP1/1,AP1/2,AP1/1_AP1/2,TFX_DMSO,TFX_DMSO_AP1/1_AP1/2
4,chr17,8148050,8148887,1,TFX2_DMSO,AP1/1,AP1/2,AP1/1_AP1/2,TFX_DMSO,TFX_DMSO_AP1/1_AP1/2


In [15]:
tmp = dat.astype({"Motif_x": str, "Motif_y": str, "Motif": str})
tmp = tmp  \
    .groupby(["Sample", "Motif_x", "Motif_y", "X"]) \
    .agg(Value=('Count_Frag', sum)) \
    .reset_index()
tmp

,Sample,Motif_x,Motif_y,X,Value
0,TFX2_DMSO,AP1/1,AP1/2,TFX_DMSO_AP1/1_AP1/2,597
1,TFX2_DMSO,AP1/1,nan,TFX_DMSO_AP1/1,87
2,TFX2_DMSO,nan,AP1/2,TFX_DMSO_AP1/2,266


**Wrappup merge**

In [16]:
def fun_motif(motif1, motif2):
    """..."""
    motif = motif1.astype(str) + "_" + motif2.astype(str)
    motif = motif.str.replace("_nan|nan_", "")
    return motif

def fun_design(sample, motif):
    """..."""
    group = sample.str.replace("[0-9]|_20x", "")
    if "Input" == group[0]:
        return "Input"
    else:
        return group.astype(str) + "_" + motif.astype(str)

In [17]:
dat = pd.merge(df1, df2, on=['Chrom_Frag', 'Start_Frag', "End_Frag", "Count_Frag", "Sample"], how='outer')
dat = dat.assign(Motif = lambda x: fun_motif(x.Motif_x, x.Motif_y))
dat = dat.assign(X     = lambda x: fun_design(x.Sample, x.Motif))

print(dat.head(3))
print(dat.tail(3))

  Chrom_Frag  Start_Frag  End_Frag  Count_Frag     Sample Motif_x Motif_y  \
0      chr17     8148000   8148888           1  TFX2_DMSO   AP1/1   AP1/2   
1      chr17     8148001   8148888           1  TFX2_DMSO   AP1/1   AP1/2   
2      chr17     8148049   8148884           1  TFX2_DMSO   AP1/1   AP1/2   

         Motif                     X  
0  AP1/1_AP1/2  TFX_DMSO_AP1/1_AP1/2  
1  AP1/1_AP1/2  TFX_DMSO_AP1/1_AP1/2  
2  AP1/1_AP1/2  TFX_DMSO_AP1/1_AP1/2  
    Chrom_Frag  Start_Frag  End_Frag  Count_Frag     Sample Motif_x Motif_y  \
754      chr17     8154275   8155119           1  TFX2_DMSO     NaN   AP1/2   
755      chr17     8154275   8155120           2  TFX2_DMSO     NaN   AP1/2   
756      chr17     8154276   8155120           1  TFX2_DMSO     NaN   AP1/2   

     Motif               X  
754  AP1/2  TFX_DMSO_AP1/2  
755  AP1/2  TFX_DMSO_AP1/2  
756  AP1/2  TFX_DMSO_AP1/2  


In [18]:
print(Counter(dat.Motif))
print(Counter(dat.Sample))
print(Counter(dat.X))

Counter({'AP1/1_AP1/2': 470, 'AP1/2': 221, 'AP1/1': 66})
Counter({'TFX2_DMSO': 757})
Counter({'TFX_DMSO_AP1/1_AP1/2': 470, 'TFX_DMSO_AP1/2': 221, 'TFX_DMSO_AP1/1': 66})


In [19]:
tmp = dat.astype({"Motif_x": str, "Motif_y": str, "Motif": str})
tmp = tmp  \
    .groupby(["Sample", "Motif_x", "Motif_y", "X"]) \
    .agg(Value=('Count_Frag', sum)) \
    .reset_index()
tmp

,Sample,Motif_x,Motif_y,X,Value
0,TFX2_DMSO,AP1/1,AP1/2,TFX_DMSO_AP1/1_AP1/2,597
1,TFX2_DMSO,AP1/1,nan,TFX_DMSO_AP1/1,87
2,TFX2_DMSO,nan,AP1/2,TFX_DMSO_AP1/2,266


## wrappup

In [124]:
from functools import reduce
import multiprocessing
print("Number of cpu : ", multiprocessing.cpu_count())

Number of cpu :  56


In [149]:
"_".join(["A", "B"])

'A_B'

In [157]:
SAMPLES = np.concatenate([
    reduce(np.char.add, ["Input", np.arange(1,6).astype(str)        ]),
    reduce(np.char.add, ["Input", np.arange(1,6).astype(str), "_20x"]),
    reduce(np.char.add, ["TFX",   np.arange(2,6).astype(str), "_DMSO"]),
    reduce(np.char.add, ["TFX",   np.arange(2,6).astype(str), "_Dex"])
])


def get_motif_data(motif_pair, sample, verbose=True):
    """..."""
    ### INIT
    lst_dat = []
    is_miss = False
    mtfs    = [motif.replace("_merge.bed.gz", "") for motif in motif_pair]
    txt     = "_".join(mtfs)
    
    for mtf, motif in zip(mtfs, motif_pair):
        ### INIT
        lst   = []
        idx   = -1 # HANDLE EXCEPTION: Empty file
        
        ### set input file path
        fdiry = os.path.join(FD_RES, "annotation_fragment", sample, TARGET)
        fname = motif
        fpath = os.path.join(fdiry, fname)
        
        ### PRINT: results of filtering
        if verbose:
            print(txt, "Import", sample, mtf, fpath)
        
        ### import data
        with gzip.open(fpath, 'rb') as file:
            for idx, line in enumerate(file):

                ### preprocess each line 
                line = str(line, 'utf-8')
                line = line.strip().split("\t")

                ### extract needed values
                idx1, idx2, idx3, idx4 = 5, 6, 8, 9
                len_mtf = int(line[idx2]) - int(line[idx1])
                len_lap = int(line[idx4])
                score   = float(line[idx3])

                ### filtering motifs scores and make sure
                ### fragments cover the full motif
                if (len_mtf <= len_lap) & (score >= THRESHOLD_MOTIF):
                    lst.append(line)

        ### PRINT: results of filtering
        if verbose:
            print(txt, "Filter", sample, mtf, str(idx + 1) + "-" + str(len(lst)), flush=True)        
    
        ### HANDLE EXCEPTION: Empty data
        if len(lst) == 0:
            if verbose:
                print(txt, "Skip Empty", sample, mtf, flush=True) 
            is_miss = True
            break
        
        ### update list
        lst_dat.append(lst)
        
    
    ### concatenate motif names
    return lst_dat, txt, is_miss

def get_motif_table(lst, sample, verbose=True):
    """..."""
    ### wrap up a list of lines into a dataframe
    CNAMES = ["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
              "Chrom_MTF",  "Start_MTF",  "End_MTF",
              "Motif",      "Score",      "Overlap"]
    CTYPES = [np.str, np.int,   np.int, np.int,
              np.str, np.int,   np.int,
              np.str, np.float, np.int]
    dat = pd.DataFrame(lst, columns=CNAMES) \
        .astype(dict(zip(CNAMES, CTYPES))) \
        .assign(Sample = sample)

    ### select columns
    dat = dat.loc[:,["Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Sample", "Motif"]]
    return dat

def fun_motif(motif1, motif2):
    """..."""
    motif = motif1.astype(str) + "_" + motif2.astype(str)
    motif = motif.str.replace("_nan|nan_", "")
    return motif

def fun_design(sample, motif):
    """..."""
    group = sample.str.replace("[0-9]|_20x", "")
    if "Input" == group[0]:
        return "Input"
    else:
        return group.astype(str) + "_" + motif.astype(str)

def get_motif_count(motif_pair, verbose=True):
    lst_res = []
    for sample in SAMPLES:
        ### get motif data
        lst_dat, txt, is_miss = get_motif_data(motif_pair, sample, verbose=verbose)
        if is_miss:
            continue

        ### get motif table
        df1 = get_motif_table(lst_dat[0], sample, verbose=verbose)
        df2 = get_motif_table(lst_dat[1], sample, verbose=verbose)
        lst_dat = None
        
        ### merge annotations
        dat = pd.merge(df1, df2, on=['Chrom_Frag', 'Start_Frag', "End_Frag", "Count_Frag", "Sample"], how='outer')
        dat = dat.assign(Motif = lambda x: fun_motif(x.Motif_x, x.Motif_y))
        dat = dat.assign(X     = lambda x: fun_design(x.Sample, x.Motif))

        ### get motif count
        dat = dat.astype({"Motif_x": str, "Motif_y": str, "Motif": str})
        dat = dat  \
            .groupby(["Sample", "Motif_x", "Motif_y", "X"]) \
            .agg(Value=('Count_Frag', sum)) \
            .reset_index()
        lst_res.append(dat)
    return pd.concat(lst_res)

In [158]:
motif_pair

('AP1_1_merge.bed.gz', 'ARI5A_merge.bed.gz')

In [159]:
sample = "TFX2_DMSO"
motif_pair     = ("AP1_1_merge.bed.gz", "AP1_2_merge.bed.gz")
motif1, motif2 = motif_pair

lst_dat, mtfs, is_miss = get_motif_data(motif_pair, sample, verbose=verbose)
print(len(lst_dat), mtfs, is_miss)

AP1_1 Import /home/mount/work/out/proj_combeffect/annotation_fragment/TFX2_DMSO/target_PER1/AP1_1_merge.bed.gz
AP1_1 Filter 400-398
AP1_2 Import /home/mount/work/out/proj_combeffect/annotation_fragment/TFX2_DMSO/target_PER1/AP1_2_merge.bed.gz
AP1_2 Filter 575-559
2 AP1_1_AP1_2 False


In [160]:
motif_pair = ("AP1_1_merge.bed.gz", "ARI5A_merge.bed.gz")
motif1, motif2 = motif_pair

lst_dat, mtfs, is_miss = get_motif_data(motif_pair, sample, verbose=verbose)
print(len(lst_dat), mtfs, is_miss)

AP1_1 Import /home/mount/work/out/proj_combeffect/annotation_fragment/TFX2_DMSO/target_PER1/AP1_1_merge.bed.gz
AP1_1 Filter 400-398
ARI5A Import /home/mount/work/out/proj_combeffect/annotation_fragment/TFX2_DMSO/target_PER1/ARI5A_merge.bed.gz
ARI5A TFX2_DMSO Skip Empty
1 AP1_1_ARI5A True


In [161]:
get_motif_table(lst_dat[0], sample)

,Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Sample,Motif
0,chr17,8148000,8148888,1,TFX2_DMSO,AP1/1
1,chr17,8148001,8148888,1,TFX2_DMSO,AP1/1
2,chr17,8148049,8148884,1,TFX2_DMSO,AP1/1
3,chr17,8148049,8148888,1,TFX2_DMSO,AP1/1
4,chr17,8148050,8148887,1,TFX2_DMSO,AP1/1
...,...,...,...,...,...,...
393,chr17,8158036,8159046,1,TFX2_DMSO,AP1/1
394,chr17,8158199,8159117,1,TFX2_DMSO,AP1/1
395,chr17,8158199,8159119,1,TFX2_DMSO,AP1/1
396,chr17,8158581,8159492,1,TFX2_DMSO,AP1/1


In [162]:
motif_pair = ("AP1_1_merge.bed.gz", "AP1_2_merge.bed.gz")
get_motif_count(motif_pair)

AP1_1 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1/AP1_1_merge.bed.gz
AP1_1 Filter 129-129
AP1_2 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1/AP1_2_merge.bed.gz
AP1_2 Filter 153-149
AP1_1 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1/AP1_1_merge.bed.gz
AP1_1 Filter 132-128
AP1_2 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1/AP1_2_merge.bed.gz
AP1_2 Filter 158-157
AP1_1 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1/AP1_1_merge.bed.gz
AP1_1 Filter 171-169
AP1_2 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1/AP1_2_merge.bed.gz
AP1_2 Filter 217-215
AP1_1 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1/AP1_1_merge.bed.gz
AP1_1 Filter 139-139
AP1_2 Import /home/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1/AP1_2_me

,Sample,Motif_x,Motif_y,X,Value
0,Input1,AP1/1,AP1/2,Input,157
1,Input1,AP1/1,nan,Input,30
2,Input1,nan,AP1/2,Input,50
0,Input2,AP1/1,AP1/2,Input,148
1,Input2,AP1/1,nan,Input,27
2,Input2,nan,AP1/2,Input,45
0,Input3,AP1/1,AP1/2,Input,174
1,Input3,AP1/1,nan,Input,42
2,Input3,nan,AP1/2,Input,82
0,Input4,AP1/1,AP1/2,Input,162


In [143]:
motif_pairs = it.combinations(MOTIFS[:3], 2)
pool = multiprocessing.Pool(1)
res  = pool.map(get_motif_count, motif_pairs)

AHR
/home/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1/AHR_merge.bed.gz
AHR Filter 76-76
AIRE
/home/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1/AIRE_merge.bed.gz
AIRE Filter 14-13
AHR
/home/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1/AHR_merge.bed.gz
AHR Filter 86-86
AIRE
/home/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1/AIRE_merge.bed.gz
AIRE Filter 14-14
AHR
/home/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1/AHR_merge.bed.gz
AHR Filter 130-130
AIRE
/home/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1/AIRE_merge.bed.gz
AIRE Filter 23-23
AHR
/home/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1/AHR_merge.bed.gz
AHR Filter 90-89
AIRE
/home/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1/AIRE_merge.bed.gz
AIRE Filter 15-15
AHR
/home/mount/work/out/proj_combeffect/annotation_fragment/Input5/target

In [144]:
len(res)

3

In [145]:
motif_pairs = it.combinations(MOTIFS[:3], 2)
list(motif_pairs)

[('AHR_merge.bed.gz', 'AIRE_merge.bed.gz'),
 ('AHR_merge.bed.gz', 'AP1_1_merge.bed.gz'),
 ('AIRE_merge.bed.gz', 'AP1_1_merge.bed.gz')]

In [146]:
res[0]

,Sample,Motif_x,Motif_y,X,Value
0,Input1,AHR,nan,Input,76
1,Input1,nan,AIRE,Input,13
0,Input2,AHR,nan,Input,86
1,Input2,nan,AIRE,Input,14
0,Input3,AHR,nan,Input,136
1,Input3,nan,AIRE,Input,23
0,Input4,AHR,nan,Input,90
1,Input4,nan,AIRE,Input,15
0,Input5,AHR,nan,Input,57
1,Input5,nan,AIRE,Input,9
